In [ ]:
from numpy import load
import numpy as np
import matplotlib.pyplot as plt
import statistics
import time
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
import argparse
from scipy.io import savemat
import h5py

# train autoencoder for classification with no compression in the bottleneck layer
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot


# 64 x 64 ADP FC-AE

## Load Data

In [ ]:
import h5py

seed = 42
test_data_size = 0.1
#augmentedADP_BS1_Nt32_Nc32_BW100MHz_O1_0p5.mat
with h5py.File('augmentedADP_BS1_Nt64_Nc64_BW100MHz_O1_3p5_0p5.mat', 'r') as file:
    # List all groups in the file
    print("Keys: %s" % file.keys())

    # Get the data from a specific dataset
    ADP = file['augADP'][:]
   # loc = file['L'][:]
    
x_train = ADP

In [ ]:
import tensorflow.keras.layers
import tensorflow.keras.models

# Encoder
x = tensorflow.keras.layers.Input(shape=(64*64), name="encoder_input")

encoder_dense_layer1 = tensorflow.keras.layers.Dense(units=32*32, name="encoder_dense_1")(x)
encoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_dense_layer1)

encoder_dense_layer2 = tensorflow.keras.layers.Dense(units=16*16, name="encoder_dense_2")(encoder_activ_layer1)
encoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_2")(encoder_dense_layer2)

encoder_dense_layer3 = tensorflow.keras.layers.Dense(units=4*4, name="encoder_dense_3")(encoder_activ_layer2)
encoder_output = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_3")(encoder_dense_layer3)

encoder = tensorflow.keras.models.Model(x, encoder_output, name="encoder_model")
encoder.summary()

# Decoder
decoder_input = tensorflow.keras.layers.Input(shape=(4*4), name="decoder_input")

decoder_dense_layer1 = tensorflow.keras.layers.Dense(units=16*16, name="decoder_dense_1")(decoder_input)
decoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_dense_layer1)

decoder_dense_layer2 = tensorflow.keras.layers.Dense(units=32*32, name="decoder_dense_2")(decoder_activ_layer1)
decoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_2")(decoder_dense_layer2)

decoder_dense_layer3 = tensorflow.keras.layers.Dense(units=64*64, name="decoder_dense_3")(decoder_activ_layer2)
decoder_output = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_3")(decoder_dense_layer3)

decoder = tensorflow.keras.models.Model(decoder_input, decoder_output, name="decoder_model")
decoder.summary()

# Autoencoder
ae_input = tensorflow.keras.layers.Input(shape=(64*64), name="AE_input")
ae_encoder_output = encoder(ae_input)
ae_decoder_output = decoder(ae_encoder_output)

ae = tensorflow.keras.models.Model(ae_input, ae_decoder_output, name="AE")
ae.summary()

## Train

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import TensorBoard


ae.compile(loss="mse", optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.0005))
tensorboard_callback = TensorBoard(log_dir='logs')

# Now, train your model
history = ae.fit(x_train, x_train,
          epochs=500,
          batch_size=32,
          shuffle=True,
          validation_split = 0.2,
          callbacks=[tensorboard_callback])


In [ ]:

ae.save('ae_64to4_O1.h5')
#encoder = keras.Model(input_img, encoded)
#decoder = keras.Model(encoded,decoded)
# loading whole model
#from keras.models import load_model
encoder.save('encoder_64to4_O1.h5')
decoder.save('decoder_64to4_O1.h5')

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()


## Test

In [ ]:
def mse(imageA, imageB):
    A = np.matrix.flatten(imageA)
    B = np.matrix.flatten(imageB)
  # the 'Mean Squared Error' between the two images is the
  # sum of the squared difference between the two images;
  # NOTE: the two images must have the same dimension
    if (np.linalg.norm(A) == 0) or (np.linalg.norm(B) == 0):
        err = 0
    else:
        err =(np.sum(np.multiply(A,B)))/(np.linalg.norm(A)*np.linalg.norm(B))
    # return the MSE, the lower the error, the more "similar"

    return err

In [ ]:
import h5py


with h5py.File('augmentedADP_BS1_Nt64_Nc64_BW100MHz_O1_3p5_0p5_test.mat', 'r') as file:
    # List all groups in the file
    print("Keys: %s" % file.keys())

    # Get the data from a specific dataset
    ADP = file['ADP_test'][:]
   # loc = file['L'][:]
x_test = ADP

In [ ]:
# Testing AE
# predict using trained encoder
x_test_orig =np.reshape(x_test,[-1,64,64])
x_test_1d =np.reshape(x_test,[-1,64*64])
encoded_images = encoder.predict(x_test_1d)
decoded_images = decoder.predict(encoded_images)
decoded_images_orig = np.reshape(decoded_images, newshape=(decoded_images.shape[0], 64,64))

In [ ]:
from scipy.io import savemat
eADP  = encoded_images
dADP = decoded_images



mdic = {"ADP": x_test ,"eADP": eADP ,"dADP": dADP }
savemat("ADP_BS1_Nt64_Nc64_BW100MHz_01_encoded.mat", mdic)

# 32 x 32 ADP FC-AE

## Load Data

In [ ]:
import h5py

seed = 42
test_data_size = 0.1
#augmentedADP_BS1_Nt32_Nc32_BW100MHz_O1_0p5.mat
with h5py.File('augmentedADP_BS1_Nt32_Nc32_BW100MHz_O1_0p5.mat', 'r') as file:
    # List all groups in the file
    print("Keys: %s" % file.keys())

    # Get the data from a specific dataset
    ADP = file['augADP'][:]
   # loc = file['L'][:]
    
x_train = ADP

## Train

In [ ]:
import tensorflow.keras.layers
import tensorflow.keras.models
import tensorflow.keras.optimizers
import tensorflow.keras.datasets
import numpy
import matplotlib.pyplot

# Encoder
x = tensorflow.keras.layers.Input(shape=(32*32), name="encoder_input")

encoder_dense_layer1 = tensorflow.keras.layers.Dense(units=16*16, name="encoder_dense_1")(x)
encoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_dense_layer1)

encoder_dense_layer2 = tensorflow.keras.layers.Dense(units=8*8, name="encoder_dense_2")(encoder_activ_layer1)
encoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_2")(encoder_dense_layer2)

encoder_dense_layer3 = tensorflow.keras.layers.Dense(units=4*4, name="encoder_dense_3")(encoder_activ_layer2)
encoder_output = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_3")(encoder_dense_layer3)



encoder = tensorflow.keras.models.Model(x, encoder_output, name="encoder_model")
encoder.summary()

# Decoder
decoder_input = tensorflow.keras.layers.Input(shape=(4*4), name="decoder_input")

decoder_dense_layer1 = tensorflow.keras.layers.Dense(units=8*8, name="decoder_dense_1")(decoder_input)
decoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_dense_layer1)

decoder_dense_layer2 = tensorflow.keras.layers.Dense(units=16*16, name="decoder_dense_2")(decoder_activ_layer1)
decoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_2")(decoder_dense_layer2)

decoder_dense_layer3 = tensorflow.keras.layers.Dense(units=32*32, name="decoder_dense_3")(decoder_activ_layer2)
decoder_output = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_3")(decoder_dense_layer3)



decoder = tensorflow.keras.models.Model(decoder_input, decoder_output, name="decoder_model")
decoder.summary()

# Autoencoder
ae_input = tensorflow.keras.layers.Input(shape=(32*32), name="AE_input")
ae_encoder_output = encoder(ae_input)
ae_decoder_output = decoder(ae_encoder_output)

ae = tensorflow.keras.models.Model(ae_input, ae_decoder_output, name="AE")
ae.summary()


In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import TensorBoard


ae.compile(loss="mse", optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.0005))
tensorboard_callback = TensorBoard(log_dir='logs')

# Now, train your model
history = ae.fit(x_train, x_train,
          epochs=500,
          batch_size=32,
          shuffle=True,
          validation_split = 0.2,
          callbacks=[tensorboard_callback])

ae.save('ae_32to4_O1.h5')
#encoder = keras.Model(input_img, encoded)
#decoder = keras.Model(encoded,decoded)
# loading whole model
#from keras.models import load_model
encoder.save('encoder_32to4_O1.h5')
decoder.save('decoder_32to4_O1.h5')

## Test 

In [ ]:
import h5py

seed = 42
test_data_size = 0.1
#augmentedADP_BS1_Nt32_Nc32_BW100MHz_O1_0p5.mat
with h5py.File('augmentedADP_BS1_Nt32_Nc32_BW100MHz_O1_0p5.mat', 'r') as file:
    # List all groups in the file
    print("Keys: %s" % file.keys())

    # Get the data from a specific dataset
    ADP = file['augADP'][:]
   # loc = file['L'][:]

In [ ]:
# Testing AE
# predict using trained encoder
x_test = ADP
x_test_orig =np.reshape(x_test,[-1,32,32])
x_test_1d =np.reshape(x_test,[-1,32*32])
encoded_images = encoder.predict(x_test_1d)
decoded_images = decoder.predict(encoded_images)
decoded_images_orig = numpy.reshape(decoded_images, newshape=(decoded_images.shape[0], 32,32))

In [ ]:

e = np.zeros(len(decoded_images_orig))
for i, image in enumerate(decoded_images_orig):
  e[i] = mse(image,x_test_orig[i,:,:])

print(np.mean(e))
print(np.std(e))

In [ ]:
import h5py

seed = 42
test_data_size = 0.1
#augmentedADP_BS1_Nt32_Nc32_BW100MHz_O1_0p5.mat
with h5py.File('augmentedADP_BS1_Nt32_Nc32_BW100MHz_O1_0p5.mat', 'r') as file:
    # List all groups in the file
    print("Keys: %s" % file.keys())

    # Get the data from a specific dataset
    ADP = file['augADP'][:]
   # loc = file['L'][:]

# 16 x 16 ADP FC-AE Model
- the load, train and test are the same as in the previous runs

In [ ]:
# Load data

In [ ]:
import tensorflow.keras.layers
import tensorflow.keras.models
import tensorflow.keras.optimizers
import tensorflow.keras.datasets
import numpy
import matplotlib.pyplot

# Encoder
x = tensorflow.keras.layers.Input(shape=(16*16), name="encoder_input")

encoder_dense_layer1 = tensorflow.keras.layers.Dense(units=12*12, name="encoder_dense_1")(x)
encoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_dense_layer1)

encoder_dense_layer2 = tensorflow.keras.layers.Dense(units=8*8, name="encoder_dense_2")(encoder_activ_layer1)
encoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_2")(encoder_dense_layer2)

encoder_dense_layer3 = tensorflow.keras.layers.Dense(units=4*4, name="encoder_dense_3")(encoder_activ_layer2)
encoder_output = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_3")(encoder_dense_layer3)



encoder = tensorflow.keras.models.Model(x, encoder_output, name="encoder_model")
encoder.summary()

# Decoder
decoder_input = tensorflow.keras.layers.Input(shape=(4*4), name="decoder_input")

decoder_dense_layer1 = tensorflow.keras.layers.Dense(units=8*8, name="decoder_dense_1")(decoder_input)
decoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_dense_layer1)

decoder_dense_layer2 = tensorflow.keras.layers.Dense(units=12*12, name="decoder_dense_2")(decoder_activ_layer1)
decoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_2")(decoder_dense_layer2)

decoder_dense_layer3 = tensorflow.keras.layers.Dense(units=16*16, name="decoder_dense_3")(decoder_activ_layer2)
decoder_output = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_3")(decoder_dense_layer3)



decoder = tensorflow.keras.models.Model(decoder_input, decoder_output, name="decoder_model")
decoder.summary()

# Autoencoder
ae_input = tensorflow.keras.layers.Input(shape=(16*16), name="AE_input")
ae_encoder_output = encoder(ae_input)
ae_decoder_output = decoder(ae_encoder_output)

ae = tensorflow.keras.models.Model(ae_input, ae_decoder_output, name="AE")
ae.summary()


In [ ]:
# Train

In [ ]:
# Test and Save

# 8 x 8 ADP FC-AE Model

In [ ]:
# Load Data

In [ ]:
import tensorflow.keras.layers
import tensorflow.keras.models
import tensorflow.keras.optimizers
import tensorflow.keras.datasets
import numpy
import matplotlib.pyplot

# Encoder
x = tensorflow.keras.layers.Input(shape=(8*8), name="encoder_input")

encoder_dense_layer1 = tensorflow.keras.layers.Dense(units=6*6, name="encoder_dense_1")(x)
encoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_dense_layer1)

encoder_dense_layer2 = tensorflow.keras.layers.Dense(units=4*4, name="encoder_dense_2")(encoder_activ_layer1)
encoder_output = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_2")(encoder_dense_layer2)


encoder = tensorflow.keras.models.Model(x, encoder_output, name="encoder_model")
encoder.summary()

# Decoder
decoder_input = tensorflow.keras.layers.Input(shape=(4*4), name="decoder_input")

decoder_dense_layer1 = tensorflow.keras.layers.Dense(units=6*6, name="decoder_dense_1")(decoder_input)
decoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_dense_layer1)

decoder_dense_layer2 = tensorflow.keras.layers.Dense(units=8*8, name="decoder_dense_2")(decoder_activ_layer1)
decoder_output = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_2")(decoder_dense_layer2)



decoder = tensorflow.keras.models.Model(decoder_input, decoder_output, name="decoder_model")
decoder.summary()

# Autoencoder
ae_input = tensorflow.keras.layers.Input(shape=(8*8), name="AE_input")
ae_encoder_output = encoder(ae_input)
ae_decoder_output = decoder(ae_encoder_output)

ae = tensorflow.keras.models.Model(ae_input, ae_decoder_output, name="AE")
ae.summary()

In [ ]:
# Train

In [ ]:
# Test and Save